In [31]:
import json
import re

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "png"
pio.templates.default = "plotly_white"

from pathlib import Path

In [24]:
BASEDIR = "../output"
RUN_PREFIX = ("20231130_215541")

In [37]:
def parse_filename(filename):
    regex = re.compile(r"[\d_]+_mu([\d.]+)_gamma([\d.]*\d+).*")
    matches = re.search(regex, filename)
    return matches[1], matches[2]

In [38]:
def read_runs(prefix, basedir):
    files = list(map(str, Path(BASEDIR).glob(prefix + "*")))
    runs = {}
    for file in files:
        mu, gamma = parse_filename(file)
        df = pd.read_csv(file)
        runs[(mu, gamma)] = df
    return runs

In [39]:
runs = read_runs(RUN_PREFIX, BASEDIR)

In [40]:
runs

{('0.1',
  '0.1'):      round  client  stage  variable      value
 0        1       5  train      loss   1.314827
 1        1       5   test      loss   1.420679
 2        1       5   test  accuracy   1.000000
 3        1       7  train      loss   3.326578
 4        1       7   test      loss   2.095891
 ..     ...     ...    ...       ...        ...
 280     10      84   test      loss  12.891831
 281     10      84   test  accuracy   0.112936
 282     10      96  train      loss   4.278706
 283     10      96   test      loss  13.248907
 284     10      96   test  accuracy   0.217391
 
 [285 rows x 5 columns],
 ('1',
  '0.1'):      round  client  stage  variable     value
 0        1       0  train      loss  0.512437
 1        1       0   test      loss  0.394369
 2        1       0   test  accuracy  0.940639
 3        1      15  train      loss  1.190833
 4        1      15   test      loss  1.480805
 ..     ...     ...    ...       ...       ...
 277     10      83   test      lo